In [1]:
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import hist
from hist import Hist
from coffea.nanoevents import NanoEventsFactory, BaseSchema
import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, schemas
import numpy as np
import mplhep as hep
from prettytable import PrettyTable


In [2]:
fileset = "/eos/home-b/bchitrod/HHH/NANOAOD/TRSM_XToHY_6b_M3_2800_M2_2300_NANOAOD.root" 

In [3]:
eventsToRead = 10000
events = NanoEventsFactory.from_root(fileset,schemaclass=NanoAODSchema,metadata={"dataset": "testSignal"},entry_stop=eventsToRead).events()

In [4]:
events.event

<Array [401, 402, 403, ... 7198, 7199, 7200] type='10000 * uint64[parameters={"_...'>

In [5]:
#useful functions
def closest(masses):
    delta = abs(125 - masses)
    closest_masses = ak.min(delta, axis=1)
    is_closest = (delta == closest_masses)
    return is_closest

def HbbvsQCD(fatjet):
    score = (fatjet.particleNetMD_Xbb/(fatjet.particleNetMD_Xbb+fatjet.particleNetMD_QCD))
    return score


In [6]:
    #Semiboosted selection
    fatjets = events.FatJet
    jets = events.Jet

    ptcut = 250
    etacut = 2.5
    mass_cut = [100,150]
    pNet_cut = 0.9105

    good_fatjets = fatjets[(fatjets.pt>ptcut) & (np.absolute(fatjets.eta)<etacut) & (fatjets.msoftdrop>=mass_cut[0]) & (fatjets.msoftdrop<=mass_cut[1])] #& (fatjets.particleNet_HbbvsQCD>=pNet_cut)                                                  
    good_semiboosted = good_fatjets[ak.num(good_fatjets, axis=1) == 2]
    good_boosted = good_fatjets[ak.num(good_fatjets, axis=1)> 2]

    res_ptcut = 30
    res_etacut = 2.5
    res_mass_cut = [90,150]
    # loose cut = 0.0532, med_cut = 0.3040, tight_cut = 0.7476 , https://twiki.cern.ch/twiki/bin/view/CMS/BtagRecommendation106XUL17                                                                                                                  
    res_deepBcut = 0.0532

    good_jets = jets[(jets.pt > res_ptcut) & (np.absolute(jets.eta) < res_etacut) & (jets.btagDeepB>res_deepBcut)]

    res_jets = (good_jets[ak.num(good_fatjets, axis=1) == 2])
    good_pairs = res_jets.nearest(good_semiboosted).delta_r(res_jets)>0.8
    good_paired = res_jets[good_pairs]
    res_bjets = (good_paired[ak.num(good_paired, axis=1) >= 2])

    dijets = ak.combinations(res_bjets, 2, fields=['i0', 'i1'])
    dijet_masses = (dijets['i0'] + dijets['i1']).mass
    is_closest = closest(dijet_masses)
    closest_dijets = dijets[is_closest]

    mass_jets = closest_dijets[((closest_dijets['i0'] + closest_dijets['i1']).mass>=res_mass_cut[0]) & ((closest_dijets['i0'] + closest_dijets['i1']).mass<=res_mass_cut[1])]
    selected_jets = mass_jets[ak.num(mass_jets,axis=1)>0]
    selected_fatjets = good_semiboosted[ak.num(selected_jets, axis=1)>0]

In [7]:
    #initiating arrays
    fj1_pt = []
    fj1_eta = []
    fj1_phi = []
    fj1_mass = []
    fj1_pnet = []
    fj2_pt = []
    fj2_eta = []
    fj2_phi = []
    fj2_mass =[]
    fj2_pnet = []
    rj1_pt = []
    rj1_eta = []
    rj1_phi = []
    rj2_pt = []
    rj2_eta = []
    rj2_phi = []
    nevnt = []
    Numbers = []


In [8]:
#interesting events 
tri_mass = (selected_fatjets[:,0]+selected_fatjets[:,1]+selected_jets['i0']+selected_jets['i1']).mass
mass_1800 = []
mass_2800 = []
for i in tri_mass:
    for j in i:
        if j>=1780.0 and j<=1820.0:
            mass_1800.append(j)
        if j>=2780.0and j<=2820.0:
            mass_2800.append(j)

In [14]:
#finding event number for interesting events
for mass in mass_1800:
    mass_fatjet = selected_fatjets[tri_mass == mass]
    event_select = events[ak.num(mass_fatjet,axis=1)>0]
    nevnt.append(event_select.event)
    
#tracing array element number from event number

for j in nevnt:
    n=0
    for i in events.event:
        if i==j:
            Numbers.append(n)
            break
        n += 1
print(Numbers)

[4, 76, 106, 168, 178, 208, 279, 405, 451, 492, 625, 639, 658, 726, 908, 1014, 1063, 4, 76, 106, 168, 178, 208, 279, 405, 451, 492, 625, 639, 658, 726, 908, 1014, 1063, 4, 76, 106, 168, 178, 208, 279, 405, 451, 492, 625, 639, 658, 726, 908, 1014, 1063]


In [10]:
    for mass,n in zip(mass_1800,Numbers):
        fj1_pt.append(fatjets[n,0].pt)
        fj1_eta.append(fatjets[n,0].eta)
        fj1_phi.append(fatjets[n,0].phi)
        fj1_mass.append(fatjets[n,0].msoftdrop)
        fj1_pnet.append(HbbvsQCD(fatjets[n,0]))
        fj2_pt.append(fatjets[n,1].pt)
        fj2_eta.append(fatjets[n,1].eta)
        fj2_phi.append(fatjets[n,1].phi)
        fj2_mass.append(fatjets[n,1].msoftdrop)
        fj2_pnet.append(HbbvsQCD(fatjets[n,1]))
        rj1_pt.append(jets[n,0].pt)
        rj1_eta.append(jets[n,0].eta)
        rj1_phi.append(jets[n,0].phi)
        rj2_pt.append(jets[n,1].pt)
        rj2_eta.append(jets[n,1].eta)
        rj2_phi.append(jets[n,1].phi)

In [11]:
def makePrettyTable(table_col1, table_col2, table_col3,table_col4,table_col5,table_col6,table_col7,table_col8,table_col9,table_col10,table_col11,table_col12,table_col14,table_col15,table_col16):
    table = PrettyTable()
    table.add_column("Event Number", table_col1)
    table.add_column("FatJet1 Pt", table_col2)
    table.add_column("FatJet1 Eta", table_col3)
    table.add_column("FatJet1 Phi", table_col4)
    table.add_column("FatJet1 Mass", table_col5)
    table.add_column("FatJet2 Pt", table_col6)
    table.add_column("FatJet2 Eta", table_col7)
    table.add_column("FatJet2 Phi", table_col8)
    table.add_column("FatJet2 Mass", table_col9)
    table.add_column("ResJet1 Pt", table_col10)
    table.add_column("ResJet1 Eta", table_col11)
    table.add_column("ResJet1 Phi", table_col12)
    table.add_column("ResJet2 Pt", table_col14)
    table.add_column("ResJet2 Eta", table_col15)
    table.add_column("ResJet2 Phi", table_col16)
    return table

In [12]:
t1 = makePrettyTable(nevnt,fj1_pt, fj1_eta, fj1_phi, fj1_mass,fj2_pt, fj2_eta, fj2_phi, fj2_mass,rj1_pt, rj1_eta, rj1_phi, rj2_pt, rj2_eta, rj2_phi)

In [13]:
print(t1)

+--------------+------------+------------------+-------------------+--------------+------------+--------------------+-------------------+------------------------+------------+------------------+-------------------+------------+-------------------+--------------------+
| Event Number | FatJet1 Pt |   FatJet1 Eta    |    FatJet1 Phi    | FatJet1 Mass | FatJet2 Pt |    FatJet2 Eta     |    FatJet2 Phi    |      FatJet2 Mass      | ResJet1 Pt |   ResJet1 Eta    |    ResJet1 Phi    | ResJet2 Pt |    ResJet2 Eta    |    ResJet2 Phi     |
+--------------+------------+------------------+-------------------+--------------+------------+--------------------+-------------------+------------------------+------------+------------------+-------------------+------------+-------------------+--------------------+
|    [405]     |   1086.0   |   0.552734375    |   2.16259765625   |   129.375    |   1016.0   |   -1.9287109375    |   -1.39013671875  |        91.4375         |   1061.0   |  0.54638671875   